## 0. Pandasのデータフレーム

ExcelやCSVなど、IRでテーブルデータを扱うことは多く、**Pandas**モジュール（ライブラリ）はテーブルデータを処理する道具として広く用いられています。CSVやExcelとの親和性も高いです。モジュールは

   **import モジュール名 as 別名**
 
で呼び、以降は別名に指定した文字列で用います。Pandasであれば、

In [ ]:
import pandas as pd # ここで実行してください、以下でpdを使用します。実行しても何も表示されません。

とすることが多いです（別名をpdとしている例が多い）。モジュールの関数を使う場合は

**pd.メソッド名( )**

のように使います。

--------

## 1. Excelの表をPythonであつかう

largeおよびsmallのフォルダには、

|テーブル|CSVファイル|
|--|--|
|学科マスタ|M_department.csv|
|コースマスタ|M_course.csv|
|科目マスタ|M_lecture.csv|
|国名マスタ|M_country.csv|
|教員情報|M_faculty.csv|
|授業主担当|D_assignment.csv|
|授業分担率|D_chief.csv|
|学籍情報|M_student.csv, M_student_small.csv|
|成績情報|D_grade.csv, D_grade_small.csv|

largeとsmallの違いは、学籍情報についてlargeはsmallの十倍、成績情報はそれに伴って調整してるところです。

これらテーブルは、Pandasモジュールの**データフレーム**というデータ構造として読み込まれています。Excelファイルから読み込むときは **.read_csv()** メソッドを使います。

In [ ]:
dpt = pd.read_csv("./large/M_department.xlsx", sheet_name='学科マスタ', engine="openpyxl", dtype={'学科コード':str})                                           
crs = pd.read_csv("./large/M_course.xlsx", sheet_name='コースマスタ', engine="openpyxl", dtype={'コースコード':str})                                           
sbj = pd.read_csv("./large/M_lecture.xlsx", sheet_name='科目マスタ', engine="openpyxl", dtype={'学科コード':str, '科目コード':str,'コースコード':str})         
nat = pd.read_csv("./large/M_country.xlsx", sheet_name='国名マスタ', engine="openpyxl", dtype={'国名コード':str})                                              
fac = pd.read_csv("./large/M_faculty.xlsx", sheet_name='教員情報', engine="openpyxl", dtype={'PID':str})                                                       
chg = pd.read_csv("./large/D_chief.xlsx", sheet_name='授業主担当', engine="openpyxl", dtype={'科目コード':str, 'PID':str})                                     
asn = pd.read_csv("./large/D_assignment.xlsx", sheet_name='授業分担率', engine="openpyxl", dtype={'科目コード':str, 'PID':str})                                
std = pd.read_csv("./large/M_student.xlsx", sheet_name='学籍情報', engine="openpyxl", dtype={'SID':str, '学科':str})                                           
grd = pd.read_csv("./large/D_grade.xlsx", sheet_name='成績情報', engine="openpyxl", dtype={'科目コード':str, 'SID':str, '主担当':str})                                                                                                     

そして、dptからgrdまでの各変数を見てみましょう。

*注）"コード"としているIDの列は数字文字列ですが、読み込むときに数値に変換するので、dtype変数に明示的に文字列データ（str）であることを指定してデータフレームに読み込んでいます。*

In [ ]:
grd

教務サンプル.xlsxの各シートが、データフレームというデータ構造で、各変数に格納されています。それぞれのカラムを見たいときは、データフレームgrd（成績情報）を例に取ると

````
grd.columns
````

か

````
list(grd.columns)
````

を実行してください。**[データフレーム名].columns** がカラム（属性）の名前を持っています。

In [ ]:
print(grd.columns)
print(list(grd.columns))

**.info()**　メソッドは、さらに詳細なデータフレームの情報を表示します。

In [ ]:
grd.info()

また、データフレームは部分列に分割することができます。

In [ ]:
sbj_tmp = sbj[['科目コード','科目名','単位数']]
sbj_tmp # sbjを評価したときに表示される結果と比較してみてください。

さらに、フィルタも次のように実行できます。以下、単位数が1である科目の抽出です。

In [ ]:
sbj_one_crd = sbj[ sbj['単位数']==1 ]
sbj_one_crd

行（番号）と列（属性）を指定して値を拾うには、.locメソッドを使います。

In [ ]:
sbj.loc[120, '科目名']

------

### 2. 教務サンプル・データベースのテーブル

<img src="https://www.irds.titech.ac.jp/wp-content/uploads/2021/05/tables.png">

一般に、業務システム（operational system）のデータベースに含まれるテーブルは**マスタ系**と**トランザクション系**に分類されます。

- マスタ系・・・・業務を遂行する上で必要な基礎データ
- トランザクション系・・・・業務を遂行すると発生するデータ

しかし、必ずしもどちらか明確に分類されるわけではないので、どちらにも解釈できる場合、ここではハイブリッド系と呼ぶことにします。

上のER図は、3.1で読み込んだ9個のテーブルのスキーム（テーブル定義）です。エンティティ（ここではテーブル）の色分けは、

- 黄色：マスタ系（ほぼ発生・更新することがない）
- 赤色：トランザクション系（学期単位で発生）
- 緑色：ハイブリッド系（年単位で発生）

としています。エンティティの矩形は3つに分かれており、上から順にテーブル名、主キー（複合の場合は全部）、属性が並んでいます。


> これらのサンプルテーブルは、**主キーに対し残りの属性がすべて関数従属する**（主キーが決まれば他の属性の値が一意にきまる）ように構成されています。逆に言うと、全てのテーブルで主キーは重複することがないようになっています。
>
> 一般に、データベース・テーブルの1レコードが何を表しているのか明確にしておく必要があります。IRでは、さまざまなデータを組み合わせて（テーブルを結合して）新しいデータテーブルを生成して分析を行います。新しく生成されたテーブルに含まれる1レコードが、何を表しているかと言うことを明確にできなければ、分析が曖昧なものとなってしまいます。このことを次のように呼びましょう。
>
> **One record, one fact. (一行一事実)**

-----

## 3. 教務サンプルのビュー

サンプルの教務システムは、データの入力・更新を行うためにテーブルを結合させた**ビュー**を持っています。

以下、データベースで定義されているビューをER図で見ていきますが、紹介するビュー群は、教務データベースが構築される際に、開発者が設計するものです。IRは、この技術情報をもとにDWを設計する必要があるので、この情報は非常に重要です。業務システム（データベース）の担当が、こうした技術情報を適切に管理しているかどうか、確認する必要があります。改修のたびに、手を加えた点や変更した点を技術情報に反映しておかなければ、IRでの活用ができなくなります。

以下のビューは、教務システムの技術情報として管理されている（と期待される）ものです。IRが管理するのではなく、システム担当が維持管理するものです。ER図ではないかもしれませんが、これら技術情報からDWに渡すためのデータテーブルを**再構築する**ことがIRの仕事の出発点になります。

1. <a href="./faculties_view.png">教員情報一覧ビュー</a>
2. <a href="./subjects_view.png">授業情報一覧ビュー</a>
3. <a href="./students_view.png">学籍情報一覧ビュー</a>
4. <a href="./charges_view.png">授業主担当一覧ビュー</a>
5. <a href="./rates_view.png">授業分担率一覧ビュー</a>
6. <a href="./grades_view.png">成績一覧ビュー</a>

### 3.a. 教員情報一覧ビュー

<img src="https://www.irds.titech.ac.jp/wp-content/uploads/2021/05/faculties_view.png">

> 教員情報テーブルは、1レコードに大学に在職する「一人」の教員の氏名、（名の）読み、性別、国籍（コード）が記されています。また、国名マスタは、1レコードに「一つの国」の国名コード、国名（英語表記）、国名（日本語表記）が記されています。国名マスタと教員情報は、それぞれ「国名コード」と「国籍」の属性で結合し、右側の「教員情報一覧ビュー」を構成することができることを示しています。ただし、国名マスタと教員情報のレコードの対応は**1対多**。

*注1）DWに教員情報一覧テーブルを作成する際、事実表（ファクトテーブル）となるエンティティ（ここでは教員情報テーブル）を紫色としました。以下、事実表となりえるエンティティを紫色にします。*

*注2）ER図で何を示すかによって、事実表や次元表は相対的に変わります。教員情報一覧ビューを構成する際には、教員情報テーブルは事実表ですが、以下に示す成績情報一覧ビューでは事実表とはならず、次元表を構成するテーブルの一つになります。*

SQLで上記ビューを生成すると次のようになると思います。

```
select 教員情報.PID, 教員情報.名前, 教員情報.読み, 国名マスタ.国名（日） as 国籍, 教員情報.性別
from 教員情報 inner join 国名マスタ 
on 教員情報.国籍 = 国名マスタ.国名コード;
```

これをpythonのpandasモジュールを使うと、以下のように実行できます。

In [ ]:
# 
# 教員情報はfac変数, 国名マスタはnat変数に読み込まれていた。
# kyoinjoho変数に結合したビューを代入することにする。
#

fac_v = pd.merge(fac, nat, left_on='国籍', right_on='国名コード')
fac_v

In [ ]:
#
# 余計な列を削除、データフレームの.dropメソッドを使う。国籍,国名コード,国名（英）が不要。columnsパラメタに削除する列名（属性）をリストで渡す。
# 結果は、kyoinjoho変数に上書き
#

fac_v = fac_v.drop(columns=['国籍','国名コード','国名（英）'])
fac_v

In [ ]:
#
# 属性（列）名である「国名（日）」を「国籍に入れ替え」。.renameメソッドを使う。属性名の変更は辞書型で与える。
#

fac_v = fac_v.rename(columns={'国名（日）':'国籍'})
fac_v

In [ ]:
#
# 以上の操作をメソッドを繋ぐことで一気に実行可能
#

fac_v = pd.merge(fac, nat, left_on='国籍', right_on='国名コード').drop(columns=['国籍','国名コード','国名（英）']).rename(columns={'国名（日）':'国籍'})
fac_v

処理結果のデータフレームはcsvやexcelに落とすことができます。

In [ ]:
fac_v.to_csv("./faculties_view.csv", index=False)
fac_v.to_excel("./faculties_view.xlsx", index=False)

# index=False は、出力するデータに新たにインデックスをつけないというオプション

----

### 3.b. 授業科目一覧ビュー

<img src="https://www.irds.titech.ac.jp/wp-content/uploads/2021/05/subjects_view.png">

（＊テーブルの順序が上下逆ですが、のちの表示の都合です）

> 科目マスタテーブルは、1レコードに、開講している「一つ」の授業の科目名、（その科目が分類されている）コース（コード）、専門（専門分野か否かを示すブール値）、単位数が記されています。また、コースマスタは、1レコードに「一つのコース」のコースコード、コース（名）が記されています。コースマスタと科目マスタは、「コースコード」属性で結合し、右側の「授業科目一覧」を構成することができることを示しています。ただし、コースマスタと科目マスタのレコードの対応は**1対多**。

結合させる属性は、両テーブルに同じ名前である場合は省略します。

**【問題】科目マスタとコースマスタは、それぞれ変数sbjと変数crsに読み込まれていますので、sbj_v変数に授業科目一覧のビューを生成してください。**

   注）結合で対応する属性が同じ名前の場合は、left_onとright_onをonで省略できます。
````
pd.merge([左のデータフレーム], [右のデータフレーム], on="共通する結合属性")
````

In [ ]:
sbj_v = pd.merge(sbj, crs, on='コースコード').drop(columns=['コースコード'])
sbj_v

----

### 3.c. 学籍情報一覧ビュー

<img src="https://www.irds.titech.ac.jp/wp-content/uploads/2021/05/students_view.png">

> 学籍情報テーブルは、1レコードに大学に所属する「一人」の学生の氏名、（名の）読み、国籍、性別、（所属する）学科コードが記されています。学科マスタは、1レコードに「一つの学科」の学科コード、学科（名）が記されています。国名マスタは、1レコードに「一つの国」の国名コード、国名（英語表記）、国名（日本語表記）が記されています。学科マスタは学籍情報と**1対多**、国名マスタと学籍情報も**1対多**。

今回はマスタを2つ参照しますが、順番にmergeします。

In [ ]:
std_v = pd.merge(std, dpt, left_on="学科",right_on="学科コード")
std_v = pd.merge(std_v, nat, left_on="国籍", right_on="国名コード")
std_v

mergeメソッドを使うとき、各データフレームに同じ名前のカラム（属性）があった場合は、第一引数（left）のカラムには'_x'、第二引数（right）のカラムには'_y'が付加される。

In [ ]:
std_v = std_v.drop(columns=["国籍","学科_x","学科コード","国名コード","国名（英）"]).rename(columns={'学科_y':'学科', '国名（日）':'国籍'})
std_v

----

### 3.d. 授業主担当一覧ビュー

<img src="https://www.irds.titech.ac.jp/wp-content/uploads/2021/05/charges_view.png">

> 一つの科目を複数の教員で担当することもある。授業主担当テーブルは、1レコードが一つの科目に対応し、主担当の教員PIDをと科目コードが記されています。授業主担当と科目マスタは**1対1**、教員情報と授業主担当は**1対多**。科目マスタとコースマスタ、教員情報と国名マスタについては、それぞれ授業科目一覧ビューと教員情報一覧ビューを参照してください。

このビューでは、すでに作成した授業科目一覧ビュー（sbj_v）と教員情報一覧ビュー（fac_v）が部分的に使われています。授業主担当ビュー（chg_v）の生成は、これらを活用します。付け替えたカラムの名前に注意しながら、mergeします。

In [ ]:
chg_v = pd.merge(chg, fac_v, on="PID").drop(columns=["PID"])
chg_v = pd.merge(chg_v,sbj_v, on="科目コード").rename(columns={'名前':'授業主担当教員','国籍':'授業主担当教員国籍'})
chg_v

----

##### **ER図とテーブルからIR用のDWテーブルを生成する際に注意すべきこと**

1. ビューを作るときに、属性名の付け替えに注意すること。間違った属性名は、先の分析でエラーを起こす可能性があります。
2. ER図では、事実表を中心に枝が伸びていく図になっていますが、**事実表から枝葉の方に多対1の方向性が保持**されていることに注意してください。途中、1対多や多対多がある場合は、生成されるデータテーブルで"**One record, one fact**"の原則が破られてしまいます。

----

### 3.d. 成績情報一覧ビュー

<img src="https://www.irds.titech.ac.jp/wp-content/uploads/2021/05/grades_view.png">

> 成績情報テーブルでは、1レコードに「一人の学生」が履修した「一つの科目」の評点が記されています。成績情報と授業主担当、科目マスタ、学籍情報は、すべて多対1となっています。授業主担当、科目マスタ、学籍情報から先は、それぞれの一覧ビューを参照。

授業主担当の部分は、授業主担当一覧ビューの一部分（科目情報が省かれている）ですが、授業主担当一覧ビューを活用することにします。

- <a href="./subjects_view.png">授業科目一覧ビュー</a>
- <a href="./students_view.png">学籍情報一覧ビュー</a>
- <a href="./charges_view.png">授業主担当一覧ビュー</a>

In [ ]:
grd_v = pd.merge(grd, std_v, on="SID")
grd_v

In [ ]:
grd_v = pd.merge(grd_v, sbj_v, on="科目コード").drop(columns=['コース'])
grd_v

In [ ]:
# 授業主担当情報一覧ビューから、次の属性は grd_v にある属性と同じ名前なので、mergeする前に削除しておく。
# ['性別','読み']....教員の属性であり、grd_vのそれらは学生の属性である。
#
chg_v = chg_v.drop(columns=['性別','読み','科目名','専門','コース','単位数'])

grd_v = pd.merge(grd_v, chg_v, on="科目コード")
grd_v

grd_vが、成績情報一覧ビューと内容となり、DW用のデータテーブルとなります。

In [ ]:
grd_v.to_excel("./subjects_view.xlsx", index=False)

----

## 4. Star Schema

教務サンプルに含まれていた既存の成績情報一覧ビューを再利用して、DW用のデータテーブルを作成しました。ER図はシステムの設計図であり、改修ごとに変わっていく可能性があります。IRがDW用のデータテーブルの設計図を残す方法として、ER図を少し抽象化したStar Schemaと呼ばれる記法で記述することがあります。

システムにはないビュー（例えば他のシステムからの連携）を作る場合にも、Star Schemaで記述しておいて、詳細はそれぞれのシステムの技術情報を参照すると言った手法をとります。

<img src="https://www.irds.titech.ac.jp/wp-content/uploads/2021/05/StarSchema.png">

<img src="https://www.irds.titech.ac.jp/wp-content/uploads/2021/05//StarSchema2.png" width="400">

# 問題

以下の授業分担率一覧ビュー（変数名はwld_v）を、Pythonを用いて構成し、Excelファイル”rates_view.xlsx”に保存してください。

-------